In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, element
import time

In [2]:
base_url = "https://www.pro-football-reference.com/players/"
list_of_players = ["Jared Goff", "Carson Wentz", "Paxton Lynch", "Christian Hackenberg", "Jacoby Brissett", "Cody Kessler", "Connor Cook", "Dak Prescott", "Cardale Jones", "Kevin Hogan", "Nate Sudfeld", "Jake Rudock", "Brandon Allen", "Jeff Driskel", "Brandon Doughty"]
pick_NO = [1,2,26,51,91,93,100,135,139,162,187,191,201,207,223]
list_of_years = ["2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
player_urls = ["G/GoffJa00/", "W/WentCa00/", "L/LyncPa00/", "H/HackCh01/", "B/BrisJa00/", "K/KessCo00/", "C/CookCo00/", "P/PresDa01/", "J/JoneCa01/", "H/HogaKe00/", "S/SudfNa00/", "R/RudoJa00/", "A/AlleBr00/", "D/DrisJe00/", "D/DougBr00/"]

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

In [4]:
stats= []
for player, pick, player_Name in zip(player_urls, pick_NO, list_of_players):
    for year in list_of_years:
        url = f"{base_url}{player}/gamelog/{year}"
        api_response = requests.get(url, headers=headers)
        time.sleep(3)
        soup = BeautifulSoup(api_response.text, 'html.parser')
        table = soup.find("tbody")
        if table:
            for row in table.find_all("tr"):
                game_row = []
                if not row.find("td", class_="center"):
                    game_row.append(2016)
                    game_row.append(player_Name)
                    game_row.append(pick)
                    
                    # Game date
                    try:
                        game_row.append(row.find("td", {"data-stat": "game_date"}).text)
                    except (AttributeError, ValueError):
                        game_row.append("")

                    # Game result
                    try:
                        game_row.append(row.find("td", {"data-stat": "game_result"}).text[0])
                    except (AttributeError, IndexError):
                        game_row.append("")

                    # Passing completions
                    try:
                        pass_cmp = row.find("td", {"data-stat": "pass_cmp"}).text
                        game_row.append(int(pass_cmp) if pass_cmp else 0)
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Passing attempts
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_att"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Passing yards
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_yds"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Passing touchdowns
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_td"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Interceptions
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_int"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Passer rating
                    try:
                        game_row.append(float(row.find("td", {"data-stat": "pass_rating"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0.0)

                    # Times sacked
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_sacked"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Rushing attempts
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "rush_att"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Rushing yards
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "rush_yds"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Rushing touchdowns
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "rush_td"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Fumbles
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "fumbles"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    stats.append(game_row)


In [7]:
df = pd.DataFrame(stats, columns=[
    'Year_Drafted', 'Player', 'Pick', 'Game_Date', 'Result', 'Pass_Cmp', 
    'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'Pass_Rating', 
    'Pass_Sacked', 'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Fumbles'
])
df.to_csv('2016.csv')